In [1]:
from utils import get_reactions
from tokenization import smi_tokenizer
from enumeration import get_alternative_smiles

In [2]:
filtered = get_reactions('../filtered.json', n = 10)

In [7]:
tokenized = get_reactions('../tokenized_reactions.json', n = 10)

In [3]:
filtered[0]

{'reaction_core': '[H] - N ( - C ( - C ) = O ) - c ( : c ) : c ( : c ) - c ( : c ) : n ( - [H] ) : c >> c : c 1 : n : c ( - C ) : n ( : c ) : c ( : c ) : c : 1 : c',
 'mapped_reaction': '[c:1]1(-[c:2]2[c:5]([N:10]([C:15]([C:18]([H:1018])([H:2018])[H:3018])=[O:19])[H:1010])[c:9]([H:1009])[c:14]([H:1014])[c:11]([H:1011])[c:6]2[H:1006])[n:3]([H:1003])[c:7]2[c:8]([c:4]1[H:1004])[c:13]([H:1013])[c:17]([H:1017])[c:16]([H:1016])[c:12]2[H:1012]>>[c:1]12[c:2]3[c:5]([c:9]([H:1009])[c:14]([H:1014])[c:11]([H:1011])[c:6]3[H:1006])[n:10][c:15]([C:18]([H:1018])([H:2018])[H:3018])[n:3]1[c:7]1[c:8]([c:4]2[H:1004])[c:13]([H:1013])[c:17]([H:1017])[c:16]([H:1016])[c:12]1[H:1012]',
 'reaxys_id': 316610,
 'meta': {'solvents': [],
  'reagents': [{'reaxys_id': 1098214,
    'molecule_name': 'hydrogenchloride',
    'smiles': '[H]Cl'}],
  'catalysts': []},
 'reactants': ['[H] c 1 c ( [H] ) c ( [H] ) c ( N ( [H] ) C ( = O ) C ( [H] ) ( [H] ) [H] ) c ( - c 2 c ( [H] ) c 3 c ( [H] ) c ( [H] ) c ( [H] ) c ( [H] ) c 

In [7]:
prod = tokenized[0]['products'][0]

In [8]:
prod_smiles = "".join(prod.split())

In [20]:
Enumerator.get_alternative_smiles([prod_smiles])

['c1cccc2nc(n3c4c(cc3c21)cccc4)C']

In [21]:
prod_smiles

'[H]c1c([H])c([H])c2c(nc(C([H])([H])[H])n3c4c([H])c([H])c([H])c([H])c4c([H])c23)c1[H]'

In [27]:
float(None)

TypeError: float() argument must be a string or a number, not 'NoneType'

In [37]:
x = 10

In [38]:
if x and x > 5:
    print('A')

A


In [22]:
from rdkit import Chem

In [90]:
m = Chem.MolToSmiles(Chem.MolFromSmiles(prod_smiles))

In [91]:
smi_tokenizer(m)

'C c 1 n c 2 c c c c c 2 c 2 c c 3 c c c c c 3 n 1 2'

In [18]:
filtered[0]["products"]

['[H] c 1 c ( [H] ) c ( [H] ) c 2 c ( n c ( C ( [H] ) ( [H] ) [H] ) n 3 c 4 c ( [H] ) c ( [H] ) c ( [H] ) c ( [H] ) c 4 c ( [H] ) c 2 3 ) c 1 [H]']

In [90]:
from copy import deepcopy
import json
from tqdm import tqdm_notebook


def detokenize_entry(entry: dict, fields: list=["reactants", "products"]):
    for field in fields:
        new = []
        for entity in entry[field]:
            new.append("".join(entity.split()))
        entry[field] = new
    return entry


def process_json_entry(entry : dict, n_copies : int = 2):
    entry = detokenize_entry(entry)
    processed = dict()
    processed["reaxys_id"] = entry["reaxys_id"]
    for i, reactant in enumerate(entry["reactants"]):
        processed[f"reactant_{i}"] = get_alternative_smiles(reactant, n_copies)
    for i, product in enumerate(entry["products"]):
        processed[f"product_{i}"] = get_alternative_smiles(product, n_copies)
    return processed


def process_file(in_fname: str, out_fname: str, processor):
    with open(in_fname, "r") as istream, open(out_fname, "w") as ostream:
        for entry in tqdm_notebook(istream):
            entry = processor(json.loads(entry))
            if entry:
                json.dump(entry, ostream)
                ostream.write("\n")

In [21]:
process_file("../filtered.json", "../enumerated.json", process_json_entry)

In [23]:
enumerated = get_reactions('../enumerated.json', 10)

In [74]:
import re

def tokenize_enum_entry(entry: dict):
    exp = re.compile(r"^(reactant|product)_\d+$")
    for key in entry.keys():
        if not exp.match(key):
            continue
        entry[key] = [smi_tokenizer(smiles) for smiles in entry[key]]
    return entry

In [79]:
process_file("../enumerated.json", "../tokenized.json", tokenize_enum_entry)

In [92]:
def get_vocab(in_fname: str):
    from collections import Counter
    exp = re.compile(r"^(reactant|product)_\d+$")
    vocab = Counter()
    with open(in_fname, 'r') as istream:
        for entry in tqdm_notebook(istream):
            entry = json.loads(entry)
            for key in entry.keys():
                if not exp.match(key):
                    continue
                for entity in entry[key]:
                    for ch in entity.split():
                        vocab[ch] += 1 
    return vocab

In [93]:
vocab = analyze_vocab("../tokenized.json")

In [101]:
sorted(vocab.items(), key=lambda x: x[1], reverse=True)

[('c', 17483646),
 ('C', 13275628),
 ('(', 9786943),
 (')', 9786943),
 ('O', 5393506),
 ('1', 5383914),
 ('=', 3109566),
 ('2', 2499586),
 ('N', 1963470),
 ('n', 1098948),
 ('3', 804604),
 ('[C@H]', 621797),
 ('[C@@H]', 607477),
 ('-', 549960),
 ('F', 534856),
 ('Cl', 324275),
 ('S', 268748),
 ('Br', 259998),
 ('4', 201240),
 ('[nH]', 166702),
 ('/', 145400),
 ('#', 134792),
 ('B', 119684),
 ('[N+]', 95918),
 ('s', 95002),
 ('o', 93784),
 ('[Si]', 88170),
 ('[O-]', 78482),
 ('[C@]', 57751),
 ('[C@@]', 57079),
 ('5', 47476),
 ('\\', 36464),
 ('I', 34254),
 ('P', 31636),
 ('[N-]', 17494),
 ('6', 10700),
 ('7', 2498),
 ('[PH]', 902),
 ('[SiH]', 640),
 ('8', 578),
 ('[IH2]', 446),
 ('9', 130),
 ('[SiH2]', 106),
 ('[PH2]', 24),
 ('%10', 18),
 ('[SH]', 14),
 ('[SiH3]', 14),
 ('[NH+]', 14)]

In [108]:
filter_set = set([k for k, v in vocab.items() if v >= 93000])

In [109]:
filter_set

{'#',
 '(',
 ')',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '=',
 'B',
 'Br',
 'C',
 'Cl',
 'F',
 'N',
 'O',
 'S',
 '[C@@H]',
 '[C@H]',
 '[N+]',
 '[nH]',
 'c',
 'n',
 'o',
 's'}

In [100]:
LEVEL1 = set(["C", "H", "O", "N", "c", "h", "o", "n"])
LEVEL2 = set(["Br", "S", "Cl", "B", "F", "br", "s", "cl", "b", "f"])
LEVEL3 = set(["Si", "I", "P", "si", "i", "p"])
LEVEL4 = set(["Sn", "Se", "Mg", "sn", "se", "mg"])

In [102]:
from functools import partial

In [117]:
def get_entry_tokenset(entry: dict)->set:
    exp = re.compile(r"^(reactant|product)_\d+$")
    result = set()
    for key in entry.keys():
        if not exp.match(key):
            continue
        for entity in entry[key]:
            for token in entity.split():
                result.add(token)
    return result


def tokenset_is_subset(superset: set, entry: dict):
    tokenset = get_entry_tokenset(entry)
    return entry if tokenset <= superset else None

In [112]:
tokenized = get_reactions("../tokenized.json", n=10)

In [118]:
get_entry_tokenset(tokenized[0]) <= filter_set

True

In [119]:
process_file("../tokenized.json",
             "../filtered_by_tokenset.json",
             partial(tokenset_is_subset, filter_set))

## Randomizing smiles

In [1]:
from rdkit import Chem
from utils import get_reactions, get_vocab_from_tokenized

In [84]:
import random
import numpy as np
from rdkit import Chem
from math import log2
import re
from copy import copy


class TokenDistribution:


    def __init__(self, tokens, freqs = None, n_tries = 20, log_freqs=False):

        if freqs is None:
            tokens, freqs = zip(*tokens.items())
            if log_freqs:
                freqs = [log2(freq) for freq in freqs]
            total = sum(freqs)
            freqs = [freq/total for freq in freqs]

        self.tokens = tokens
        self.freqs = freqs
        self.n_tries = n_tries # Number of tries to get a character different from previous
        # Deducing optimal value of tries from freqs seems possible
        # It may be token dependent function, such as expected value of geometric distribution
        # Another option is to make self.tokens a set and sample from set difference
        # self.tokens \ {old_token}


    def sample(self, orig_token=None):
        # Works only in Python 3.6+
        new_token = random.choices(self.tokens, self.freqs)[0]
        i = 0
        while new_token == orig_token and i < self.n_tries:
            new_token = random.choices(self.tokens, self.freqs)[0]
            i += 1
        return new_token


    def sample_(self, orig_token=None):
        # Using numpy instead of standard random
        new_token = np.random.choice(self.tokens, self.freqs)[0]
        i = 0
        while new_token == orig_token and i < self.n_tries:
            new_token = random.choices(self.tokens, self.freqs)[0]
            i += 1
        return new_token


    def sample_single_insertion(self) -> str:
        # Just a single token sampled unconditionally
        return self.sample()


    def sample_insertion(self, length, mode=0) -> [str]:
        # with length parameter
        if mode == 0:
            tokens = []
            prev_token = None
            for _ in range(length):
                curr = self.sample(prev_token)
                prev_token = curr
                tokens.append(curr)
        else:
            tokens = [self.sample(None) for _ in range(length)]
        return tokens


class SmilesRandomizer:


    def __init__(self, distr, p_sub, p_in, p_del, n_tries=10):
        self.distr = distr
        self.p_sub = p_sub
        self.p_in = p_in
        self.p_del = p_del
        self.n_tries = n_tries # Number of tries to get incorrect string


    @staticmethod
    def is_correct(seq: str):
        return Chem.MolFromSmiles(seq) is not None


    def mutate(self, seq : list) -> (list, int, int, int):
        length = len(seq)
        subs, ins, dels = 0, 0, 0
        new_seq = []
        i = 0
        while i < length:
            throw = random.random()
            if throw <= self.p_sub:
                # Substitution
                success = False
                for j in range(self.n_tries):
                    char = self.distr.sample(seq[i])
                    temp = "".join(new_seq + [char] + seq[i+1:])
                    if not self.is_correct(temp):
                        new_seq.append(char)
                        success = True
                        break
                if success:
                    subs += 1
                    i += 1
                    continue
            if throw > self.p_sub and throw <= self.p_sub + self.p_in:
                # Insertion
                success = False
                for j in range(self.n_tries):
                    char = self.distr.sample(seq[i])
                    temp = "".join(new_seq  + [char] + seq[i:])
                    if not self.is_correct(temp):
                        new_seq.append(char)
                        new_seq.append(seq[i])
                        success = True
                        break
                if success:
                    ins += 1
                    i += 1
                    continue
            if throw > self.p_sub + self.p_in and throw <= self.p_sub + self.p_in + self.p_del:
                # Deletion
                temp = "".join(new_seq + seq[i+1:])
                if not self.is_correct(temp):
                    dels += 1
                    i += 1
                    continue
            new_seq.append(seq[i])
            i += 1
        return new_seq, subs, ins, dels
        
        
def randomize_entry(randomizer: SmilesRandomizer, entry: dict):
    exp = re.compile(r"^(reactant|product)_\d+$")
    for key in entry.keys():
        if not exp.match(key):
            continue
        new_field = []
        for entity in entry[key]:
            print(entity)
            new_entity, subs, ins, dels = randomizer.mutate(entity.split()) # Think about splits and joins
            new_field.append(temp)
        entry[field+'_MUTATED'] = new_field
    return entry

In [128]:
vocab = get_vocab_from_tokenized("../filtered_by_tokenset.json")

In [129]:
td = TokenDistribution(vocab, log_freqs=True)

In [130]:
randomizer = SmilesRandomizer(td, 0.05, 0.05, 0.05)

In [23]:
reacts = get_reactions('../filtered_by_tokenset.json', 10)

In [60]:
tstr = reacts[0]['product_0'][0].split()

In [70]:
mtstr = randomizer.mutate(tstr)

In [71]:
print(mtstr)
print(tstr)

(['c', '1', 'c', 'c', '2', 'c', '(', 'n', 'c', '(', 'n', '3', 'c', 'c', 'c', '3', 'c', '2', 'c', 'F', 'c', '2', ')', 'C', ')', 'c', '1'], 1, 1, 4)
['c', '1', 'c', 'c', '2', 'c', '(', 'n', 'c', '(', 'n', '3', 'c', '2', 'c', 'c', '2', 'c', '3', 'c', 'c', 'c', 'c', '2', ')', 'C', ')', 'c', 'c', '1']


In [55]:
testing[-4] = '('

In [72]:
SmilesRandomizer.is_correct("".join(mtstr[0]))

False

In [132]:
import json
import re
from tqdm import tqdm_notebook


def make_tab_line(*args):
    args = [str(x) if not isinstance(x, str) else x for x in args]
    return "\t".join(args) + "\n"

def make_randomized_sample(in_fname: str, out_fname: str, randomizer):
    exp = re.compile(r"^(reactant|product)_\d+$")
    with open(in_fname, "r") as istream, open(out_fname, "w") as ostream:
        ostream.write(make_tab_line("reaxys_id", "field_name", "original", "erroneous", "subs", "ins", "dels"))
        for entry in tqdm_notebook(istream):
            entry = json.loads(entry)
            for key in entry.keys():
                if not exp.match(key):
                    continue
                for entity in entry[key]:
                    new_ent, subs, ins, dels = randomizer.mutate(entity.split())
                    line = make_tab_line(entry['reaxys_id'],
                                         key,
                                         entity,
                                         " ".join(new_ent),
                                         subs,
                                         ins,
                                         dels)
                    ostream.write(line)

In [133]:
make_randomized_sample("../filtered_by_tokenset.json", "train_sample.tsv", randomizer)

In [117]:
import pandas as pd

In [123]:
df = pd.read_csv("train_sample.tsv", sep="\t")

In [126]:
df.loc[0, "original"]

'c 1 c 2 c ( c c ( - c 3 c ( N C ( C ) = O ) c c c c 3 ) [nH] 2 ) c c c 1'